### Start import neccessary libraries

In [1]:
import pandas as pd
import requests
from finnomena_api import finnomenaAPI
from finnomena_api.keys import keys
import warnings
import yfinance as yf
import subprocess
import time

api = finnomenaAPI()

##### First, we define the funds from which we will extract the necessary data.

In [2]:
#Create search list for Finnomena API
#Read the CSV file, the CSV file will contain the funds code-name which we are tracking
df_search_list = pd.read_csv("./csv/input.csv")

#The code below selects column 'code-name' and converts it into a "list"
code_name_column= df_search_list['code-name'].tolist()

#Then, we use the for loop to remove unnecessary string characters.
search_list = []
for i in range(len(code_name_column)):
    list = code_name_column[i].strip()
    search_list.append(list)
#To ensure that string is correct for the subsequent search.
print('we have {0} string for {1}'.format(len(search_list[0]),search_list[0])) 
#To confirming the number of code-name
print('we have {0} name-code to search'.format(len(search_list))) 


#Now we create serach list for Yahoo API
df_search_list = pd.read_csv("./csv/input.csv")
code_name_column= df_search_list['y-code'].tolist()

search_list_fed = []
for i in range(len(code_name_column)):
    list = code_name_column[i].strip()
    search_list_fed.append(list)
print('we have {0} string for {1}'.format(len(search_list_fed[0]),search_list_fed[0])) 
print('we have {0} name-code to search'.format(len(search_list_fed)))


we have 10 string for ONE-UGG-RA
we have 48 name-code to search
we have 10 string for 0P00018JE1
we have 48 name-code to search


### Start Extracting
#### Using Finnomena API to extract data 

In the end we will transform retrieved data into 3 columns
* Finnomena Security Code (Fund Code)
* NAV Date
* NAV (Current Price)

In [3]:
#Create a variable to store the unfindable search codename.
missing = []
df_funds_info = pd.DataFrame()

#Utilize a for loop to traverse each search code-name, append it to a dataframe, and then save it as a csv file.
with warnings.catch_warnings(): # suppress warning
    warnings.simplefilter('ignore')
    for i in range(len(search_list)):
        try:
            run_index = search_list[i]
            dict_info = api.get_fund_info(run_index)
            df_add = pd.DataFrame(data=[dict_info])
            df_funds_info = df_funds_info.append(df_add,ignore_index=True)
        except:
            try:
                #Storing the code-name that cannot be found, then setting all dictionary values to 0 and adding them to the dataframe as remarks.
                print('cannot find: ', run_index)
                missing.append(run_index)
                dict_info =dict.fromkeys(dict_info, '')
                dict_info['security_name'] = str(run_index)
                df_add = pd.DataFrame(data=[dict_info])
                # df_funds_info = df_funds_info.append(df_add,ignore_index=True)
                df_funds_info = pd.concat([df_funds_info, df_add], ignore_index=True)
            except:
                print('Caught Error')
        print('processing ', str(run_index))
print('finish processing')
print('missing funds is ', missing)
print('missing funds SUM', len(missing))
print('total number of funds = ',len(df_funds_info))
print('current final reviewed nav_date: ', df_funds_info['nav_date'].max())

processing  ONE-UGG-RA
processing  KFGG-A
processing  TMBGQG
processing  K-WORLDX
processing  K-USXNDQ-A(A)
processing  K-US500X-A(A)
processing  K-EUX
processing  K-EUSMALL
processing  K-JPX-A(A)
processing  K-ASIAX
processing  TMBAGLF
processing  K-CHX
processing  KT-ASHARES-A
processing  K-CHINA-A(A)
processing  P-CGREEN
processing  K-VIETNAM
processing  PRINCIPAL VNEQ-A
processing  K-INDX
processing  KFHHCARE-A
processing  K-CHANGE-A(A)
processing  KKP SEMICON-H
processing  KT-FINANCE-A
processing  UGIS-N
processing  K-CASH
processing  UOBID
processing  UOBSD
processing  K-GOLD-A(A)
processing  K-OIL
processing  KT-MINING
processing  KS50LTF-C(L)
processing  KDLTF-C(L)
processing  K-USA-SSF
processing  K-CHINA-SSF
processing  K-CHANGE-SSF
processing  KFLTFA50-D
processing  KFLTFDIV
processing  KFGGSSF
processing  UGIS-SSF
processing  BIG CAP-D LTF
processing  VALUE-D LTF
processing  KT-CHINABOND-A
processing  KT-ENERGY
processing  PCASH
processing  KUSARMF
processing  B-INNOTECHRMF

#### At this point will still need one more columns which is "fund_type", we will use beautifulsoup for that purpose

In [4]:
#The code below will retrieve the fund type from the Finnomena website using the web scraping method.
from bs4 import BeautifulSoup
fund_types_list =[]
for i in search_list:
    try:
        url = "https://www.finnomena.com/fund/" + str(i)
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        soup = soup.find_all('div', class_='detail-row')
        fund_type_add = soup[2].find('div', class_='right').text
        fund_types_list.append(fund_type_add)
    except:
        pass
    print('extracting fund_types:', str(i))
print('finish extraction')
print('total number extract fund_types is: ', len(fund_types_list))

fund_types_df = pd.DataFrame(fund_types_list)
fund_types_df = fund_types_df.rename(columns = {0: 'fund_type'})

#join fund_types to the df_funds_info dataframe
df_funds_info = pd.concat([df_funds_info, fund_types_df], axis = 1)

extracting fund_types: ONE-UGG-RA
extracting fund_types: KFGG-A
extracting fund_types: TMBGQG
extracting fund_types: K-WORLDX
extracting fund_types: K-USXNDQ-A(A)
extracting fund_types: K-US500X-A(A)
extracting fund_types: K-EUX
extracting fund_types: K-EUSMALL
extracting fund_types: K-JPX-A(A)
extracting fund_types: K-ASIAX
extracting fund_types: TMBAGLF
extracting fund_types: K-CHX
extracting fund_types: KT-ASHARES-A
extracting fund_types: K-CHINA-A(A)
extracting fund_types: P-CGREEN
extracting fund_types: K-VIETNAM
extracting fund_types: PRINCIPAL VNEQ-A
extracting fund_types: K-INDX
extracting fund_types: KFHHCARE-A
extracting fund_types: K-CHANGE-A(A)
extracting fund_types: KKP SEMICON-H
extracting fund_types: KT-FINANCE-A
extracting fund_types: UGIS-N
extracting fund_types: K-CASH
extracting fund_types: UOBID
extracting fund_types: UOBSD
extracting fund_types: K-GOLD-A(A)
extracting fund_types: K-OIL
extracting fund_types: KT-MINING
extracting fund_types: KS50LTF-C(L)
extracting 

### Start Extracting current price from Yahoo Site

In [5]:
#Utilize a for loop to traverse each search y_code, append it to a dataframe, and then save it as a csv file.
y_fund_df = pd.DataFrame({'y_code':[],'y_nav_date':[],'y_nav':[],'y_link':[]})
y_main_link = 'https://finance.yahoo.com/quote/'
with warnings.catch_warnings(): # suppress warning
    warnings.simplefilter('ignore')
    for i in search_list_fed:
        try:
            #Retrieve data element using API
            fund = yf.Ticker(str(i))
            fund_his = fund.history(period="max", interval="1d")

            #Transforming data by rounding to two decimal places, resetting the index, and then selecting only 'Date' and 'Close' columns.
            #Initially, the index is date, but using.reset index, date becomes a column so that it can be selected.
            fund_add = fund_his.round(2).sort_index(ascending=False)
            fund_add.reset_index(inplace=True)
            fund_add = fund_add[['Date','Close']]

            #We formatted the 'Date' column as a date since we do not require time for fund management.
            fund_add['Date'] = fund_add['Date'].dt.date
            
            #Create variables for y-code string, navigation, and navigation date.
            y_code = str(i)
            y_nav = fund_add['Close'].iloc[0]
            y_nav_date = fund_add['Date'].iloc[0]
            y_link = y_main_link + str(i)
            print('y-code: {0} nav: {1} & nav_date: {2}'.format(str(i),y_nav, y_nav_date))

            #Add y-code string, nav, and nav date in a dataframe
            y_fund_df = y_fund_df.append({'y_code':y_code,'y_nav_date':y_nav_date,'y_nav':y_nav,'y_link':y_link}, ignore_index = True)
        except:
            #In the event that a search is not a fed fund, we will pass the value NaN to a dataframe.
            try:
                y_code = str(i)
                y_nav = ('none')
                y_nav_date = ('none')
                y_link = ('none')
                print('THIS IS NOT FOUND y-code: {0} nav: {1} & nav_date: {2}'.format(str(i),y_nav, y_nav_date))
                y_fund_df = y_fund_df.append({'y_code':y_code,'y_nav_date':y_nav_date,'y_nav':y_nav,'y_link':y_link}, ignore_index = True)
            except:
                print('Error Caught')
                break
print('Finish extract nav and nav_date from yahoo site')

y-code: 0P00018JE1 nav: 25.78 & nav_date: 2023-02-03
y-code: 0P00018JE1 nav: 25.78 & nav_date: 2023-02-03
y-code: 0P0000TJ9P nav: 36.51 & nav_date: 2023-02-03
y-code: ACWI nav: 91.1 & nav_date: 2023-02-06
y-code: QQQ nav: 303.59 & nav_date: 2023-02-06
y-code: IVV nav: 411.8 & nav_date: 2023-02-06
y-code: EXW1.DE nav: 42.35 & nav_date: 2023-02-06
y-code: 0P0001DK0W.F nav: 16.29 & nav_date: 2023-02-06
y-code: 1306.T nav: 2074.5 & nav_date: 2023-02-07
y-code: AAXJ nav: 69.22 & nav_date: 2023-02-06
y-code: 0P00000TKE nav: 49.32 & nav_date: 2023-02-06
y-code: 82822.HK nav: 12.44 & nav_date: 2023-02-07
y-code: 0P0001IM1D nav: 1336.46 & nav_date: 2023-02-06
y-code: 0P000085UN nav: 123.2 & nav_date: 2023-02-03
y-code: KGRN nav: 30.75 & nav_date: 2023-02-06
Got error from yahoo api for ticker NONE, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- NONE: No timezone found, symbol may be delisted
THIS IS NOT FOUND y-code: none nav: none & nav_date: none
Got er

### Start Combining Dataframe from Multiple Sourcese

In [6]:
#Now we select the required columns and rename them for future use.
with warnings.catch_warnings(): # suppress warning
    warnings.simplefilter('ignore')
    df_funds_tracking = df_funds_info[['security_name','nav_date','current_price','fund_type','feeder_fund']]
    df_funds_tracking['mainlink'] = "https://www.finnomena.com/fund/"
    df_funds_tracking['weblink'] = df_funds_tracking['mainlink']+ df_funds_tracking['security_name']
    df_funds_tracking = df_funds_tracking[['security_name','nav_date','current_price','fund_type','feeder_fund','weblink']]
    df_funds_tracking = df_funds_tracking.rename(columns = {'current_price':'nav'})
    df_funds_tracking[['y_code','y_nav','y_nav_date','y_link']] = y_fund_df[['y_code','y_nav','y_nav_date','y_link']]

### Export nav data as .csv

In [7]:
#Export file to CSV
print('export file: df_funds_tracking.csv')
df_funds_tracking.to_csv('df_funds_tracking.csv',index=False)

import time
time.sleep(1)

#Open the file
import subprocess
subprocess.Popen(['start', 'df_funds_tracking.csv'], shell=True)

print('finish')

export file: df_funds_tracking.csv
finish


### Excel Automation

In [15]:
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
import pandas as pd

to_add_df =pd.read_csv('df_funds_tracking.csv')
to_add_df = to_add_df[['security_name','nav_date','nav']]

file_name = '../port-management.xlsx'
wb = load_workbook(file_name)
row_num = pd.read_csv('df_funds_tracking.csv')

row_num = len(row_num)
print('data type: ',type(row_num))
print('data len: ', row_num)

number_of_columns = to_add_df.shape[1]
print('number of columns: ', number_of_columns)

ws = wb['NAV Data']
print('Work book = ', ws)


# This is fixed code to manipulate a certain excel fiel
# 'col' is the starting columns
# 'last_col' is the last column
#****** I USE WHILE LOOP HERE BECAUSE USING get_column_letter within the for loop will cause error due to the library itself
col_excel_start = 2 #starting col >>> [1] =  column A [2] = column B [2] (in excel file it us column a as index)
col_offset = col_excel_start #for py to offset in range wqhen use iloc
last_col = number_of_columns+2
xl_col = []
while col_excel_start <= last_col:
    add = get_column_letter(col_excel_start)
    xl_col.append(add)
    col_excel_start+=1

col= 2 #re assign starting col  to >>> 2 = B
xl_col #this store the list of columns and excel we need (as string)



#To skip the header we set start_row to 2
start_row = 2
row_offset = start_row #for py as we don't want title
for row in range(start_row,row_num):
    for colxl,colpy in zip(xl_col,range(col,last_col)):
        if not pd.isna(to_add_df.iloc[row - row_offset, 2]):
            ws[colxl + str(row)].value = to_add_df.iloc[row-row_offset,colpy-col_offset]
            print(to_add_df.iloc[row-row_offset,colpy-col_offset])
        else:
            continue


wb.save(file_name)

data type:  <class 'int'>
data len:  48
number of columns:  3
Work book =  <Worksheet "NAV Data">
ONE-UGG-RA
2023-02-03
23.286
KFGG-A
2023-02-03
5.6645
TMBGQG
2023-02-03
16.7168
K-WORLDX
2023-02-03
11.2523
K-USXNDQ-A(A)
2023-02-03
23.1137
K-US500X-A(A)
2023-02-03
9.3916
K-EUX
2023-02-03
16.0742
K-EUSMALL
2023-02-03
12.9162
K-JPX-A(A)
2023-02-06
15.2822
K-ASIAX
2023-02-03
7.6138
TMBAGLF
2023-02-03
13.7382
K-CHX
2023-02-06
11.6128
KT-Ashares-A
2023-02-03
8.6194
K-CHINA-A(A)
2023-02-03
6.7171
P-CGREEN
2023-02-03
8.2998
K-VIETNAM
2023-02-06
11.6067
PRINCIPAL VNEQ-A
2023-02-06
10.7344
K-INDX
2023-02-03
13.9716
KFHHCARE-A
2023-02-03
13.344
K-CHANGE-A(A)
2023-02-03
19.0225
KKP SEMICON-H
2023-02-03
9.2282
KT-FINANCE-A
2023-02-03
24.4716
UGIS-N
2023-02-03
10.8116
K-CASH
2023-02-06
13.3108
UOBID
2023-02-06
11.9204
UOBSD
2023-02-06
12.9939
K-GOLD-A(A)
2023-02-06
11.624
K-OIL
2023-02-03
5.3338
KT-MINING
2023-02-03
7.4596
KS50LTF-C(L)
2023-02-06
12.1043
KDLTF-C(L)
2023-02-06
16.901
K-USA-SSF
2023-0

# Start retrieving process of historical data to plot candlestick and timeseries chart

#### How to Retrieve historical price from Yahoo Finance (Single Security)
<details><summary><font color="yellow">Click here for the solution</font></summary>

```python
##### Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max Either Use period parameter or use start and end
name_y = '0P00018JE1'
fund_y = yf.Ticker(name_y)
his_y = fund_y.history(period="max", interval="1d")
his_y = his_y.round(2).sort_index(ascending=False)
his_y.reset_index(inplace=True)
his_y['Date'] = his_y['Date'].dt.date
his_y.to_excel('his_y_'+str(name_y)+'.xlsx')
##### print(his_y)
```

</details>

#### How to retrieve historical price from Finnomena (Single Security)

<details><summary><font color="yellow">Click here for the solution</font></summary>

```python
name_f = 'TMBGQG'
fund_f = api.get_fund_price(name_f)
his_f = fund_f.round(2).sort_index(ascending=False)
his_f.set_index('date', inplace=True)
print(his_f)
##### his_f.reset_index(inplace=True)
##### his_f = his_f[['date','price']]
his_f.to_csv('his_f_'+str(name_f)+'.csv')
```

</details>



#### Resample from Day interval to Weekly interval for a single column dataframe (Single Security)

<details><summary><font color="yellow">Click here for the solution</font></summary>

```python
import pandas as pd
###### load data into a DataFrame
df = pd.read_csv('his_f_ONE-UGG-RA - Copy.csv', parse_dates=['date'])
###### set 'date' column as the DataFrame's index
df.set_index('date', inplace=True)
###### group data by week and take the mean of each group
df_weekly = df.resample('W-TUE').max()
df_weekly = df_weekly.round(2).sort_index(ascending=False)
```

</details>

#### How to resameple data from Daily interval into Weekly interval: YAHOO FINANCE (Single Security)

<details><summary><font color="yellow">Click here for the solution</font></summary>

```python
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max Either Use period parameter or use start and end
name_y = '0P00018JE1'
fund_y = yf.Ticker(name_y)
df = fund_y.history(period="max", interval="1d")
df = df.round(2).sort_index(ascending=False)
df.reset_index(inplace=True)



# Set the date column as the index
df.set_index('Date', inplace=True)
df.index = pd.to_datetime(df.index)

# Resample the data to weekly frequency, using the 'Close' column as the value
df_weekly = df.resample('W-MON').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last'})
df_weekly = df_weekly.rename(columns = {'Close':'Close Feeder'})
df_weekly[['Close','Code', 'Security Filter']]=['',name_y,name_y]
df_weekly['Date'] = df_weekly.index
df_weekly = df_weekly[['Open','High','Low','Close','Close Feeder','Code','Security Filter','Date']]

# Print the weekly data
df_weekly = df_weekly.round(2).sort_index(ascending=False)
# df_weekly.to_csv('df_weekly'+str(name))
print(df_weekly)
```


</details>

#### Resample from daily interval into weekly interval: Finnomena (Single Security)

<details><summary><font color="yellow">Click here for the solution</font></summary>


````python
name = 'ONE-UGG-RA'
fund_f = api.get_fund_price(name)
df = fund_f.round(2).sort_index(ascending=False)
df.set_index('date', inplace=True)
df.to_csv('his_f_'+str(name)+'.csv')
df.index = pd.to_datetime(df.index)


# Resample the data into weekly intervals
df_weekly = df.resample("W-MON").agg({"price": ["first", "max", "min", "last"]})

# Rename the columns
df_weekly.columns = ["Open", "High", "Low", "Close"]
df_weekly = df_weekly.round(2).sort_index(ascending=False)
# df_weekly.to_csv('df_weekly'+str(name)+'.csv')
df_weekly[['Close Feeder','Code', 'Security Filter']]=['',name,name]
print(df_weekly)
````


</details>

### Create Search list to filter only necessary funds which has fed a fund

In [8]:
#The procedure was the same as when we created a search list for NAV data, but this time we restricted the search list to global funds.

df_search_list = pd.read_csv("./csv/input.csv")

to_drop = []
for i in range(len(df_search_list)):
    if df_search_list.iloc[i,1] == 'none' or df_search_list.iloc[i,1] == 'to add':
        to_drop.append(i)
df_search_list = df_search_list.drop(df_search_list.index[to_drop])
search_list = df_search_list.iloc[:,0].tolist()
search_list_fed = df_search_list.iloc[:,1].tolist()

for i in range(len(search_list)):
    search_list[i] = search_list[i].strip()
print('we have {0} string for {1}'.format(len(search_list[0]),search_list[0])) 
print('we have {0} name-code to search'.format(len(search_list)))

for i in range(len(search_list_fed)):
    search_list_fed[i] = search_list_fed[i].strip()
print('we have {0} string for {1}'.format(len(search_list_fed[0]),search_list_fed[0])) 
print('we have {0} name-code to search'.format(len(search_list_fed)))

we have 10 string for ONE-UGG-RA
we have 30 name-code to search
we have 10 string for 0P00018JE1
we have 30 name-code to search


### Resample from daily interval into weekly interval: Finnomena

The data in dataframe contain only close price so we perform some transformation.
1. Create a new column titled 'Open' that will contain the oldest price value within a weekly interval.
2. Create a new column titled "High" that will contain the highest price value within a weekly interval.
3. Create a new column titled "Low" that will contain the lowest price value from the "price" column within the weekly interval.
4. Create a new column titled "Close" that contains the most recent value from the "price" column within a weekly interval.

In [9]:
#Utilize a for loop to append each historical price the Resample the data from daily interval into weekly interval

df_funds_price = pd.DataFrame()
missing =[]
with warnings.catch_warnings(): # suppress warning
    warnings.simplefilter('ignore')
    for i in range(len(search_list)):
        #FOR HISTORYCAL DATA INFORMATION
        try:
            run_index = search_list[i]
            fund_f = api.get_fund_price(run_index)
            df = fund_f.round(2).sort_index(ascending=False)

            #Set date as an index
            df.set_index("date", inplace=True)

            #Reformat the index as datetime type
            df.index = pd.to_datetime(df.index)

            # Resample the data into weekly intervals
            df_weekly = df.resample("W-MON").agg({"price": ["first", "max", "min", "last"]})

            # Rename the columns 'first', 'max', 'min', and 'last'
            df_weekly.columns = ["Open", "High", "Low", "Close Feeder"]
            
            df_weekly = df_weekly.sort_index(ascending=False)
            # df_weekly.to_csv('df_weekly'+str(name)+'.csv')
            df_weekly[['Close Fed','Code', 'Security Filter']]=['NaN',run_index,run_index]
            df_weekly['Date']=df_weekly.index
            df_weekly['Date'] = df_weekly['Date'].dt.date
            df_weekly['Year'] = pd.to_datetime(df_weekly['Date']).dt.year
            df_weekly['Categories'] = 'Local'
            df_funds_price = pd.concat([df_funds_price, df_weekly], ignore_index=True)
            print('processed:{0} {1} '.format(i,run_index))
        except:
            try:
                print('Cannot find: ', run_index)
                missing.append(run_index)
                df_weekly =dict.fromkeys(df_weekly, 'Error')
                df_weekly['code'] = str(run_index)
                df_weekly = pd.DataFrame(data=[df_weekly])
                # df_funds_info = df_funds_info.append(df_add,ignore_index=True)
                df_funds_price = pd.concat([df_funds_price, df_weekly], ignore_index=True)
            except:
                print('Caught Error')
print('missing funds is ', missing)
print('missing funds SUM', len(missing))
print('total number of funds = ',len(df_funds_price))

processed:0 ONE-UGG-RA 
processed:1 KFGG-A 
processed:2 TMBGQG 
processed:3 K-WORLDX 
processed:4 K-USXNDQ-A(A) 
processed:5 K-US500X-A(A) 
processed:6 K-EUX 
processed:7 K-EUSMALL 
processed:8 K-JPX-A(A) 
processed:9 K-ASIAX 
processed:10 TMBAGLF 
processed:11 K-CHX 
processed:12 KT-ASHARES-A 
processed:13 K-CHINA-A(A) 
processed:14 P-CGREEN 
processed:15 K-INDX 
processed:16 KFHHCARE-A 
processed:17 K-CHANGE-A(A) 
processed:18 KKP SEMICON-H 
processed:19 KT-FINANCE-A 
processed:20 UGIS-N 
processed:21 K-GOLD-A(A) 
processed:22 K-OIL 
processed:23 KT-MINING 
processed:24 KT-ENERGY 
processed:25 KUSARMF 
processed:26 B-INNOTECHRMF 
processed:27 KFGGRMF 
processed:28 KT-ASHARES RMF 
processed:29 KGARMF 
missing funds is  []
missing funds SUM 0
total number of funds =  8605


### Resameple data from Daily interval into Weekly interval: YAHOO FINANCE

1. The 'Open' column will contain the price from the 'Close' column on the earliest date within the weekly interval.
2. The 'High' column will contain the highest price from 'Close' column with in the weekly interval
3. The 'Low' column will contain the lowest price from 'Close' column within the weekly interval
4. The 'Close' column will contain the latest price within its own column within the weekly interval

In [11]:
#Utilize a for loop to append each historical price the Resample the data from daily interval into weekly interval

df_funds_price_f = pd.DataFrame()
missing =[]
with warnings.catch_warnings(): # suppress warning
    warnings.simplefilter('ignore')
    for i in range(len(search_list_fed)):
        #FOR HISTORYCAL DATA INFORMATION
        try:
            # Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max Either Use period parameter or use start and end
            run_index = search_list_fed[i]
            run_index_filter = search_list[i]
            fund_y = yf.Ticker(run_index)
            df = fund_y.history(period="max", interval="1d")
            if len(df) == 0:
                print('Cannot find: ', run_index)
                pass
            else:
                df = df.round(2).sort_index(ascending=False)
                df.reset_index(inplace=True)
                # Set the date column as the index
                df.set_index('Date', inplace=True)
                df.index = pd.to_datetime(df.index)
                # Resample the data to weekly frequency, using the 'Close' column as the value
                df_weekly = df.resample('W-MON').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last'})
                df_weekly = df_weekly.rename(columns = {'Close':'Close Fed'})
                df_weekly[['Close Feeder','Code', 'Security Filter','Categories']]=['NaN',run_index,run_index_filter,'Global']
                df_weekly['Date'] = df_weekly.index
                df_weekly['Date'] = df_weekly['Date'].dt.date
                df_weekly['Year'] = pd.to_datetime(df_weekly['Date']).dt.year
                df_weekly = df_weekly[['Open','High','Low','Close Feeder','Close Fed','Code','Security Filter','Date','Categories','Year']]
                # Print the weekly data
                df_weekly = df_weekly.round(2).sort_index(ascending=False)
                # df_weekly.to_csv('df_weekly'+str(name))
                df_funds_price_f = pd.concat([df_funds_price_f, df_weekly], ignore_index=True)
                print('processed:{0} {1} '.format(i,run_index))
        except:
            print('Caught Error')
            break
print('missing funds is ', missing)
print('missing funds SUM', len(missing))
print('total number of funds = ',len(df_funds_price_f))

processed:0 0P00018JE1 
processed:1 0P00018JE1 
processed:2 0P0000TJ9P 
processed:3 ACWI 
processed:4 QQQ 
processed:5 IVV 
processed:6 EXW1.DE 
processed:7 0P0001DK0W.F 
processed:8 1306.T 
processed:9 AAXJ 
processed:10 0P00000TKE 
processed:11 82822.HK 
processed:12 0P0001IM1D 
processed:13 0P000085UN 
processed:14 KGRN 
processed:15 INDY 
processed:16 0P0000LZRE 
processed:17 0P0001BDIT.L 
processed:18 SOXX 
processed:19 0P00000SRT.F 
processed:20 0P0000X9O0 
processed:21 GLD 
processed:22 DBO 
processed:23 0P0000SUYI.F 
processed:24 0P00000JZX 
processed:25 0P0000JM6Q 
processed:26 0P00019SEJ 
processed:27 0P00018JE1 
processed:28 0P0001IM1D 
processed:29 0P00000AWK 
missing funds is  []
missing funds SUM 0
total number of funds =  11719


### Combine historical data

In [12]:
df_funds_price_union = pd.concat([df_funds_price, df_funds_price_f], ignore_index=True)
df_funds_price_union.dropna(inplace=True)

### Cleaning Data Before put to use

In [13]:
print(df_funds_price_union.isna().sum())
print(df_funds_price_union.dtypes)
df_funds_price_union['Open'] = df_funds_price_union['Open'].astype(float)
df_funds_price_union['High'] = df_funds_price_union['High'].astype(float)
df_funds_price_union['Low'] = df_funds_price_union['Low'].astype(float)
df_funds_price_union['Close Feeder'] = df_funds_price_union['Close Feeder'].astype(float)
df_funds_price_union['Close Fed'] = df_funds_price_union['Close Fed'].astype(float)
print(df_funds_price_union.dtypes)
print(len(df_funds_price_union))
df_funds_price_union.columns

Open               0
High               0
Low                0
Close Feeder       0
Close Fed          0
Code               0
Security Filter    0
Date               0
Year               0
Categories         0
dtype: int64
Open               float64
High               float64
Low                float64
Close Feeder        object
Close Fed           object
Code                object
Security Filter     object
Date                object
Year                 int64
Categories          object
dtype: object
Open               float64
High               float64
Low                float64
Close Feeder       float64
Close Fed          float64
Code                object
Security Filter     object
Date                object
Year                 int64
Categories          object
dtype: object
20309


Index(['Open', 'High', 'Low', 'Close Feeder', 'Close Fed', 'Code',
       'Security Filter', 'Date', 'Year', 'Categories'],
      dtype='object')

In case you want to export as csv

````python
df_funds_price_union.to_csv('df_funds_price_union.csv')
subprocess.Popen(['start', 'df_funds_price_union.csv'], shell=True)
````

### Export the historical data to google sheet which will linked to Tableau dashboard

In [14]:
import pandas as pd
import pygsheets
GSHEET_NAME = 'makeone'

df = df_funds_price_union

creds = 'pkong-credential.json'
api = pygsheets.authorize(service_file=creds)
wb = api.open(GSHEET_NAME)
# open the sheet by name
sheet = wb.worksheet_by_title(f'Sheet1')
sheet.clear(start='A1', end=None, fields='*')
sheet.set_dataframe(df, (1,1))

[Link Google Sheet](https://docs.google.com/spreadsheets/d/1kpQeTokjN-G-gPOpmZbaLK0LNpuDZUzqTYNEFiu-6OU/edit#gid=0)

In [27]:
from IPython.display import IFrame

url = 'https://public.tableau.com/app/profile/pongpisut.kongdan/viz/FundsDashboard_16752377846780/Dashboard1'
IFrame(src=url, width=800, height=800)


[Link Tableau]('https://public.tableau.com/app/profile/pongpisut.kongdan/viz/FundsDashboard_16752377846780/Dashboard1')